To crawl https://www.gov.sg/factually/page-2 then 
https://www.gov.sg/factually/content/are-more-foreigners-taking-away-jobs-of-singaporeans

reference https://www.digitalocean.com/community/tutorials/how-to-scrape-web-pages-with-beautiful-soup-and-python-3

Post activity thoughts:
The code should be written for robust-ness. I understand why json is one of the the preferred communciation format, and why dictionaries are useful. Even with missing information a dictionary-like file can still be created without hassle.

In [1]:
import requests
from bs4 import BeautifulSoup
import unicodedata
import numpy as np
import time
import pandas as pd

### Define function for `threading`

In [2]:
def get_articles_from_page(page_num, debug = False):
    '''
    given page_num
    returns category_name_texts, 
            category_name_links, 
            article_title_links,
            article_title_texts
    '''
    time.sleep(np.random.uniform(0,3))
    
    category_name_texts = []
#     category_name_links = []
    article_title_texts = []
    article_title_links = []
    
    if True:
#     try:
        page = requests.get('https://www.gov.sg/factually/page-{}'.format(page_num))
        soup = BeautifulSoup(page.text, 'html.parser')

#         soup = soup.find(class_="content factually-section")
        # narrow down to this section
#         print(soup)

        grid_items = soup.find_all(class_="news-info")
#         print(grid_items)
        for grid_item in grid_items:

#             try:
                category_item = grid_item.find(class_="category")
                category_name_text = category_item.text
                category_name_text = category_name_text.strip()
#                 category_name_link = category_item.get('href')

                article_title_item = grid_item.find('h2')
                article_title_text = article_title_item.text
                article_title_link = article_title_item.find('a').get('href')

                category_name_texts.append(category_name_text)
#                 category_name_links.append(category_name_link)
                article_title_texts.append(article_title_text)
                article_title_links.append(article_title_link)

#             except:
#                 print(page_num, "error")
                
        print(str(page_num) + " ", end="")

#     except:
#         print(page_num, "ERROR")
    
    return (category_name_texts, 
#             category_name_links, 
            article_title_texts,
            article_title_links)

In [3]:
# test one out
get_articles_from_page(30)

30 

(['Health', 'Defence & Security', 'Education & Employment', 'Health'],
 ['What help will I receive when I am sick?',
  'When can I call an SCDF Ambulance?',
  'How can I remain employable?',
  "What's this MERS-CoV that’s been in the news lately?"],
 ['/factually/content/what-help-will-i-receive-when-i-am-sick',
  '/factually/content/when-can-i-call-an-scdf-ambulance',
  '/factually/content/how-can-i-remain-employable',
  '/factually/content/whats-this-merscov-thats-been-in-the-news-lately'])

### Crawling with `threading`

In [4]:
import urllib.request as urllib2
from multiprocessing.dummy import Pool as ThreadPool 
import time

start_time = time.time()

# make the Pool of workers
pool = ThreadPool(20) 

# open the urls in their own threads
# and return the results
results = pool.map(get_articles_from_page, range(1,51))  # CHANGE THIS

# close the pool and wait for the work to finish 
pool.close()
pool.join()

print(time.time() - start_time)

14 5 10 12 18 1 15 16 19 17 11 8 13 3 23 6 20 9 22 2 4 7 34 29 30 33 27 38 32 28 24 39 37 26 43 21 36 42 40 35 41 25 47 31 48 46 45 49 50 44 10.15442967414856


### Process results

In [5]:
import numpy as np
print(np.shape(results))

category_name_texts = []
# category_name_links = []
article_title_texts = []
article_title_links = []

for result in results:
    category_name_texts += result[:][0]
#     category_name_links += result[:][1]
    article_title_texts += result[:][1]
    article_title_links += result[:][2]

links_crawled_2D_array = []

for a,b,c in zip(category_name_texts,
#                    category_name_links,
                   article_title_texts,
                   article_title_links):
    links_crawled_2D_array.append([a,b,c])

my_df = pd.DataFrame(links_crawled_2D_array)
my_df.to_csv('links_crawled.csv', index=False, header=["category_name_texts", 
#                                                        "category_name_links",
                                                       "article_title_texts",
                                                       "article_title_links"])

(50, 3, 4)


In [6]:
links_crawled = pd.read_csv('links_crawled.csv', header=[0])
print(links_crawled.head())
print(links_crawled.tail())
# headlines = [headline for headline in articles["headline"][:]]

      category_name_texts                                article_title_texts  \
0        Law & Government  Does our Prime Minister get paid up to $4.5 mi...   
1  Education & Employment  Are more foreigners taking away jobs of Singap...   
2                  Health  Will the Government take back unused MediSave ...   
3             Environment  Can the prices in the 1962 Water Agreement be ...   
4  Education & Employment  MOM clarifies inaccuracies in TWC2’s articles ...   

                                 article_title_links  
0  /factually/content/does-our-prime-minister-get...  
1  /factually/content/are-more-foreigners-taking-...  
2  /factually/content/will-the-government-take-ba...  
3  /factually/content/can-the-prices-in-the-1962-...  
4      /factually/content/mom-clarifies-inaccuracies  
      category_name_texts                                article_title_texts  \
195      Law & Government  Is the Government's procurement process really...   
196     Economy & Finance  Wh

# Crawling for individual articles

In [7]:
links_crawled_matrix = [links_crawled.iloc[i] for i in range(len(links_crawled.index))]

In [8]:
# some usage
print(links_crawled.iloc[20])
# print(links_crawled.iloc[20].name)
# print(list(links_crawled["article_title_links"]))
# print(links_crawled.as_matrix)

category_name_texts                                     Law & Government
article_title_texts    5 Things to Note About the Keppel Offshore and...
article_title_links    /factually/content/5-things-to-note-about-the-...
Name: 20, dtype: object


In [9]:
def get_info_from_link(link_crawled):
    article_link = 'https://www.gov.sg' + link_crawled["article_title_links"]
    print(str(link_crawled.name) + " ", end="") # id actually

    if True:
#     try:
    #     print(article_link)
        time.sleep(np.random.uniform(0,10))
        page = requests.get(article_link)
        soup = BeautifulSoup(page.text, 'html.parser')
#         print(soup)

        headline_item = soup.find(class_="title")
        headline = headline_item.contents[0]  
#         print(headline)

        date_item = soup.find(class_="op-modified")
        date = date_item.text
    #     date = date_item.select_one("span").text  # the same
#         print(date)

        article_text = ""
        text_soup = soup.find("article")
#         print(text_soup)
        text_paras = text_soup.select("p")  # searches <p>s, may have issues on some pages
        for text_para in text_paras[:-1]:
            text_line = text_para.text
            article_text += text_line
            article_text += "\n"
    #     print(article_text)

        article_text = unicodedata.normalize("NFKD", article_text)
        # https://stackoverflow.com/questions/10993612/python-removing-xa0-from-string

        if not headline or not date or not article_text:
            print("FAULTY ", str(link_crawled.name), " ", article_link, headline, "|", date, "|", article_text)

#         time.sleep(np.random.rand())
        

        
        return [link_crawled["category_name_texts"],
#                 link_crawled["category_name_links"],
                link_crawled["article_title_texts"],
                link_crawled["article_title_links"],
                headline,
                date,
                article_text]
        
#     except:
#         print(article_link)
#         return [link_crawled["category_name_texts"],
# #                 link_crawled["category_name_links"],
#                 link_crawled["article_title_texts"],
#                 link_crawled["article_title_links"],
#                 link_crawled["article_title_texts"],
#                 date,
#                 "Text not found."]

In [10]:
get_info_from_link(links_crawled_matrix[16])

16 FAULTY  16   https://www.gov.sg//factually/content/did-the-public-transport-council-quietly-axe-the-off-peak-pass-for-seniors 
                Did the Public Transport Council “quietly” axe the “Off Peak Pass for Seniors”?
             | 
                    20 Jul 2018
                 | 


['Transport & Motoring',
 'Did the Public Transport Council “quietly” axe the “Off Peak Pass for Seniors”?',
 '/factually/content/did-the-public-transport-council-quietly-axe-the-off-peak-pass-for-seniors',
 '\r\n                Did the Public Transport Council “quietly” axe the “Off Peak Pass for Seniors”?\r\n            ',
 '\r\n                    20 Jul 2018\r\n                ',
 '']

### Crawl each individual link with `threading`

In [11]:
import urllib.request as urllib2
from multiprocessing.dummy import Pool as ThreadPool 
import time

start_time = time.time()

# make the Pool of workers
pool = ThreadPool(10) 

# open the urls in their own threads
# and return the results
results = pool.map(get_info_from_link, links_crawled_matrix[:])

# close the pool and wait for the work to finish 
pool.close()
pool.join()

print(time.time() - start_time)

0 5 10 15 20 25 30 35 40 45 46 31 11 36 26 1 16 21 41 6 47 12 32 7 48 2 13 37 FAULTY  16   https://www.gov.sg//factually/content/did-the-public-transport-council-quietly-axe-the-off-peak-pass-for-seniors 
                Did the Public Transport Council “quietly” axe the “Off Peak Pass for Seniors”?
             | 
                    20 Jul 2018
                 | 
17 27 3 22 42 49 8 FAULTY  32   https://www.gov.sg//factually/content/do-hdb-flat-buyers-own-their-flat 
                Do HDB flat buyers own their flat?
             | 
                    05 Feb 2018
                 | 
33 FAULTY  17   https://www.gov.sg//factually/content/is-it-true-our-public-universities-reserve-20-per-cent-of-their-places-for-foreign-students 
                Is it true our public universities reserve 20% of their places for foreign students?
             | 
                    02 Mar 2018
                 | 
18 43 38 14 4 28 23 50 44 9 FAULTY  18   https://www.gov.sg//factually/content/is-it-true-t

In [12]:
# print(results)

In [13]:
import pandas as pd
my_df = pd.DataFrame(results)
my_df.to_csv('articles.csv', index=False, header=["category_name_texts", 
#                                                   "category_name_links",
                                                  "article_title_texts",
                                                  "article_title_links",
                                                  "headline",
                                                  "date",
                                                  "article_text"])

In [15]:
articles = pd.read_csv('articles.csv', header=[0])
print(articles)

           category_name_texts  \
0             Law & Government   
1       Education & Employment   
2                       Health   
3                  Environment   
4       Education & Employment   
5                       Health   
6              Savings & Taxes   
7            Economy & Finance   
8            Economy & Finance   
9         Transport & Motoring   
10           Economy & Finance   
11           Economy & Finance   
12            Law & Government   
13        Transport & Motoring   
14        Transport & Motoring   
15            Law & Government   
16        Transport & Motoring   
17      Education & Employment   
18      Education & Employment   
19             Savings & Taxes   
20            Law & Government   
21             Savings & Taxes   
22             Savings & Taxes   
23        Transport & Motoring   
24                      Health   
25            Law & Government   
26        Transport & Motoring   
27                      Others   
28            

In [ ]:
# okay it seems that I got blocked from spamming the site :/